In [1]:
## Import python library
import pandas as pd
import nltk

## Read file
file_name = '/content/Redmi Note 11.csv'
## Read file using pandas
df = pd.read_csv(file_name)

In [2]:
# imports
import re
# function to remove special characters
def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(pat, '', text)

In [3]:
import re
l = []
asd_train = []
pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]'
for i in range(df.shape[0]):
    
    g = str(df.iloc[i]['body'])
    g = g.replace('[^\w\s]','')
    g = remove_special_characters(g)
    g = g.replace("/"," ")
    g = g.replace("."," ").replace(","," ").replace("\'","").replace("\"","").replace(":"," ")
    g = " ".join(g.split())
    k = ""
    for j in g:
        if(j == " "):
            k = k + j
        elif(ord(j)>=ord('A') and ord(j)<=ord('Z')):
            k = k + j
        elif(ord(j)>=ord('a') and ord(j)<=ord('z')):
            k = k + j
                
    g1 = df.iloc[i]['rating']
    if(g1>3):
        asd_train.append(1)
    else:
        asd_train.append(0)
    l.append(k)

In [4]:
df['body1'] = l

In [5]:
df['rating'] = asd_train

In [6]:
df.head()

,product,title,rating,body,body1
0,"Redmi Note 11 (Horizon Blue, 6GB RAM, 128GB St...",Not worth buying it.,0,I had redmi 6 Pro and exchanged it for Redmi 1...,I had redmi Pro and exchanged it for Redmi B...
1,"Redmi Note 11 (Horizon Blue, 6GB RAM, 128GB St...",Budget Friendly,1,For 12500/- price it's a good picScreen Amoled...,For price its a good picScreen Amoled Sounds...
2,"Redmi Note 11 (Horizon Blue, 6GB RAM, 128GB St...",Good but not great,1,"Pros- Amoled Screen, Battery easily lasts a da...",Pros Amoled Screen Battery easily lasts a day ...
3,"Redmi Note 11 (Horizon Blue, 6GB RAM, 128GB St...",Redmi Note 11,1,"Redmi Note 11 is great piece, but automatic ca...",Redmi Note is great piece but automatic call ...
4,"Redmi Note 11 (Horizon Blue, 6GB RAM, 128GB St...",Value for money,1,The media could not be loaded.\n ...,The media could not be loaded A good fone in t...


In [7]:
df.rating.value_counts()

1    845
0    590
Name: rating, dtype: int64

In [15]:
import nltk
nltk.download('stopwords')
  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
stop = nltk.corpus.stopwords.words('english')

In [20]:
## Tokenizing sentence into token for finding synonym.
def make_tokenizer(texts):
    from keras.preprocessing.text import Tokenizer
    t = Tokenizer()
    t.fit_on_texts(texts)
    return t

tokenizer = make_tokenizer(df['body1'])    ## Message is column name

X = tokenizer.texts_to_sequences(df['body1'])

from keras_preprocessing.sequence import pad_sequences
X = pad_sequences(X, 70)

In [21]:
## Dictionary of word index
index_word = {}
for word in tokenizer.word_index.keys():
    index_word[tokenizer.word_index[word]] = word

In [22]:
## word list
words = [value for key, value in index_word.items()]

In [23]:
## Function to find synonym of words 
import spacy
nlp = spacy.load('en_core_web_sm',disable=['parser'])
def check_lemma(t,w) :
    r = [d for d in t if (nlp(d.text)[0].lemma_ != nlp(w.text)[0].lemma_)]
    return r

def get_word_synonym(word):
    filtered_words = [w for w in word.vocab if (not w.lower_ in stop) and w.is_lower == word.is_lower and w.prob >= -15] ## (not w.lower_ in stop) and
    similarity = sorted(filtered_words, key=lambda w: word.similarity(w), reverse=True)
    filtered_similarity = check_lemma(similarity[:30], word)
    return filtered_similarity[:3]

/usr/local/lib/python3.7/dist-packages/spacy/language.py:1899: UserWarning: [W123] Argument disable with value ['parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  config_value=config["nlp"][key],


In [24]:
## Synonym dictionary
synonym_dict = {}

for word in words:
    synonym_dict.update({word : tuple([w.lower_ for w in get_word_synonym(nlp.vocab[word])])})
        #print(word, " : ", [w.lower_ for w in get_word_synonym(nlp.vocab[word])])

In [25]:
## Only consider filtered synonym
import collections
value_occurrences = collections.Counter(synonym_dict.values())

filtered_synonym = {key: value for key, value in synonym_dict.items() if value_occurrences[value] == 1}

In [26]:
## Function for augmenting data by replacing words with synonym using spaCy
## This might not be best best method compared to data augmentation using language translation

import re
import random
sr = random.SystemRandom()
split_pattern = re.compile(r'\s+')
def data_augmentation(message, aug_range=1) :
    augmented_messages = []
    for j in range(0,aug_range) :
        new_message = ""
        for i in filter(None, split_pattern.split(message)) :
            new_message = new_message + " " + sr.choice(filtered_synonym.get(i,[i]))
        augmented_messages.append(new_message)
    return augmented_messages

In [27]:
## Dictionary for intent count
## Intent is column name
intent_count = df.rating.value_counts().to_dict()

In [28]:
## Get max intent count to match other minority classes through data augmentation
import operator
max_intent_count = max(intent_count.items(), key=operator.itemgetter(1))[1]

In [29]:
## Loop to interate all messages
import numpy as np
import math
import tqdm
newdf = pd.DataFrame()
for intent, count in intent_count.items() :
    count_diff = max_intent_count - count    ## Difference to fill
    multiplication_count = math.ceil((count_diff)/count)  ## Multiplying a minority classes for multiplication_count times
    if (multiplication_count) :
        old_message_df = pd.DataFrame()
        new_message_df = pd.DataFrame()
        for message in tqdm.tqdm(df[df["rating"] == intent]["body1"]) :
            ## Extracting existing minority class batch
            dummy1 = pd.DataFrame([message], columns=['body1'])
            dummy1["rating"] = intent
            old_message_df = old_message_df.append(dummy1)
            
            ## Creating new augmented batch from existing minority class
            new_messages = data_augmentation(message, multiplication_count)
            dummy2 = pd.DataFrame(new_messages, columns=['body1'])
            dummy2["rating"] = intent
            new_message_df = new_message_df.append(dummy2)
        
        ## Select random data points from augmented data
        new_message_df=new_message_df.take(np.random.permutation(len(new_message_df))[:count_diff])
        
        ## Merge existing and augmented data points
        newdf = newdf.append([old_message_df,new_message_df])
    else :
        newdf = newdf.append(df[df["rating"] == intent])

100%|██████████| 590/590 [00:01<00:00, 312.84it/s]


In [30]:
newdf.rating.value_counts()

1    845
0    845
Name: rating, dtype: int64

In [31]:
newdf.head()

,product,title,rating,body,body1
1,"Redmi Note 11 (Horizon Blue, 6GB RAM, 128GB St...",Budget Friendly,1,For 12500/- price it's a good picScreen Amoled...,For price its a good picScreen Amoled Sounds...
2,"Redmi Note 11 (Horizon Blue, 6GB RAM, 128GB St...",Good but not great,1,"Pros- Amoled Screen, Battery easily lasts a da...",Pros Amoled Screen Battery easily lasts a day ...
3,"Redmi Note 11 (Horizon Blue, 6GB RAM, 128GB St...",Redmi Note 11,1,"Redmi Note 11 is great piece, but automatic ca...",Redmi Note is great piece but automatic call ...
4,"Redmi Note 11 (Horizon Blue, 6GB RAM, 128GB St...",Value for money,1,The media could not be loaded.\n ...,The media could not be loaded A good fone in t...
5,"Redmi Note 11 (Horizon Blue, 6GB RAM, 128GB St...","Good display and body , worst camera",1,Really good device with great looks and screen...,Really good device with great looks and screen...


In [32]:
newdf.tail()

,product,title,rating,body,body1
0,NaN,NaN,0,NaN,Worst Experience
0,NaN,NaN,0,NaN,I wouldnt have purchased this phone if I knew...
0,NaN,NaN,0,NaN,Nice performance
0,NaN,NaN,0,NaN,Display is niceBut the camera is too bad
0,NaN,NaN,0,NaN,camera clearty not good money waste


In [33]:
newdf.to_csv("Redmi Note 11 Balanced.csv")

In [34]:
from google.colab import files
files.download('Redmi Note 11 Balanced.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>